# A Corpus-Based Acoustic Analysis of Monophthong Vowels among Chinese Learners and Native Speakers of English - Part 3

Martin Schweinberger (`r format(Sys.time(), '%Y-%m-%d')`)


## Introduction

## Preparation

load packages


In [ ]:
library(tidyverse)
library(here)
library(adehabitatHR)
library(lme4)
library(sjPlot)
library(report)
library(flextable)
library(cowplot)      
library(randomForest) 
library(rms)    
library(caret) 
library(Hmisc) 
library(quanteda)  
#library(glmulti) 
library(partykit)   
library(ggparty)
library(hunspell)
library(janitor)
# set options
options(stringsAsFactors = F)                           
options(scipen = 999) 
options(max.print=10000)


## Load data



In [ ]:
# load .rda data
cdat  <- base::readRDS(file = here::here("data", "cleandat.rda")) %>%
  dplyr::ungroup()
# inspect
str(cdat); head(cdat)


## Reduce data



In [ ]:
bdat <- cdat %>%
  dplyr::mutate(label = stringr::str_remove_all(label, ":"),
                gender = ifelse(gender == "f", "female", gender),
                gender = ifelse(gender == "m", "male", gender),
                tvariety = ifelse(tvariety == "us", "AmE", tvariety),
                tvariety = ifelse(tvariety == "gb", "BrE", tvariety)) %>%
#  dplyr::filter(label != "A",
#                label != "O") %>%
  droplevels(.)  %>%
  dplyr::rename(Vowel = label,
                Word = word,
                TargetVariety = tvariety,
                Gender = gender,
                Duration = duration,
                Proficiency = prof,
                Speaker = speaker) %>%
  # clean word
  dplyr::mutate(Word = str_remove_all(Word, "\\W")) %>%
  dplyr::filter(hunspell_check(Word) == T) %>%
  # remove "shits"
  dplyr::filter(Word != "shits",
                Word != "stat",
                Word != "whats")
# inspect
head(bdat); names(table(bdat$Word))


## Check frequency of words



In [ ]:
# create a vector of words
words <- names(table(bdat$Word))
# load ace files
afiles <- list.files(here::here("ACE"), pattern = ".TXT", recursive = T, full.names = T) 
bfiles <- list.files(here::here("BROWN"), pattern = ".TXT", recursive = T, full.names = T) 
lfiles <- list.files(here::here("LOB"), pattern = ".TXT", recursive = T, full.names = T) 
cfiles <- c(afiles, bfiles, lfiles)


In [ ]:
# load the files by scanning the content
controlc <- sapply(cfiles, function(x){
  x <- scan(x, what = "char",  sep = "", quote = "",  quiet = T,  skipNul = T)
  x <- paste0(x, sep = " ", collapse = " ")
  x <- stringr::str_squish(x)
})
controlc <- paste0(controlc, collapse = " ")
# inspect
str(controlc)


extract word count of control corpus



In [ ]:
cleancontrolc <- controlc %>%
  stringr::str_replace_all("<.*?>", " ") %>%
  stringr::str_replace_all("[^[:alpha:] ]", " ") %>%
  stringr::str_squish() %>%
  quanteda::tokenize_fastestword() %>%
  unlist() %>%
  length()
# inspect
cleancontrolc


check how frequent the words are in the control corpus



In [ ]:
freqs <- sapply(words, function(x){
  x <- stringr::str_count(controlc, paste0("\\W", x, "\\W", sep = "", collapse = ""))
})
# convert into data frame
freqsdf <- data.frame(names(freqs), freqs, cleancontrolc) %>%
  dplyr::rename(Word = 1,
                all = 3) %>%
  dplyr::mutate(Frequency = log(freqs/all*1000)) %>%
  dplyr::select(-freqs, -all)
# inspect
head(freqsdf)


## Annotate word class



In [ ]:
lexical <- c("bad",  "bed", "best", "big", "bit", "book", "books", "boost", "boots", "boss", "bought", "buds", "bus", "butts", "dad", "dead", "death", "debt", "debts", "desk", "dish",  "dust", "gap", "gas",  "good",  "guess", "head", "heads",  "hit", "hot", "key", "kid", "kids", "pass", "past", "pat", "path", "pub", "pubs", "push", "sad", "said", "sat", "says", "seat", "seats", "see", "seep", "sees", "set", "sets",  "shits", "shoes", "shop", "shops", "shut", "sit", "skip",  "speak", "spots", "stat", "step", "steps", "stop", "stops", "stud", "suit", "task", "tasks", "tea", "teeth", "test", "tests", "took", "top", "tough", "two", "wash", "ways",  "weak", "weed", "week",  "wish",  "wood")
bdat <- bdat %>%
  dplyr::mutate(WordClass = ifelse(Word %in% lexical, "lexical", "grammatical"),
                Word = as.vector(Word))
bdat <- left_join(bdat, freqsdf, by = "Word")
# inspect
table(bdat$WordClass); head(bdat)


## Check durations



In [ ]:
bdat %>%
  dplyr::mutate(Vowel = dplyr::case_when(Vowel == "{" ~ "\u00E6",
                                         Vowel == "6" ~ "\u0250",
                                         Vowel == "e" ~ "\u0065",
                                         Vowel == "E" ~ "\u025B",
                                         Vowel == "i" ~ "\u0069",
                                         Vowel == "I" ~ "\u026A",
                                         Vowel == "Q" ~ "\u0252",
                                         Vowel == "u" ~ "\u0075",
                                         Vowel == "U" ~ "\u028A",
                                         Vowel == "V" ~ "\u028C",
                                         TRUE ~ Vowel))  %>%
  ggplot(aes(x = Vowel, y = Duration)) +
  geom_boxplot()


Remove items with exaggerated duration



In [ ]:
nrow(bdat)
bdat <- bdat  %>%
  # remove rare words
  dplyr::group_by(type, Word) %>%
  dplyr::mutate(freq = n()) %>%
#  dplyr::mutate(Word = ifelse(freq > 10, Word, "other")) %>%
  dplyr::ungroup()
# harmonize words
nnwords <- bdat %>%
  dplyr::filter(type == "CHN") %>%
  dplyr::group_by(Word) %>%
  dplyr::summarise(Freq = n()) %>%
  dplyr::pull(Word)

# remove rare vowels
bdat <- bdat %>%
  dplyr::group_by(Vowel) %>%
  dplyr::mutate(fr = n()) %>%
  dplyr::filter(fr > 100) %>%
  dplyr::select(-fr) %>%
  dplyr::ungroup()
# inspect
str(bdat); nrow(bdat)


In [ ]:
bdat %>%
  dplyr::mutate(Vowel = dplyr::case_when(Vowel == "{" ~ "\u00E6",
                                         Vowel == "6" ~ "\u0250",
                                         Vowel == "e" ~ "\u0065",
                                         Vowel == "E" ~ "\u025B",
                                         Vowel == "i" ~ "\u0069",
                                         Vowel == "I" ~ "\u026A",
                                         Vowel == "Q" ~ "\u0252",
                                         Vowel == "u" ~ "\u0075",
                                         Vowel == "U" ~ "\u028A",
                                         Vowel == "V" ~ "\u028C",
                                         TRUE ~ Vowel))  %>%
  ggplot(aes(x = Vowel, y = Duration)) +
  geom_boxplot()


In [ ]:
tb2 <- bdat %>%
  dplyr::ungroup() %>%
  dplyr::mutate(Age = dplyr::case_when(Age < 30 ~ "18-29",
                                       Age < 40 ~ "30-39",
                                       Age < 50 ~ "40-49",
                                       Age > 49 ~ "50+",
                                       TRUE ~ "unknown")) %>%
  dplyr::group_by(type, Gender, Age) %>%
  dplyr::summarise(speakers = length(table(Speaker))) %>%
  tidyr::spread(Age, speakers) %>%
  dplyr::ungroup()  %>%
  adorn_totals("row")%>%
  adorn_totals("col")
# save
write.table(tb2, here::here("tables", "tb2_icame.txt"), sep = "\t")
# inspect
tb2


In [ ]:
bdat <- bdat %>%
  dplyr::mutate(F1 = as.vector(scale(F1)),
                F2 = as.vector(scale(F2)),
                Duration = as.vector(scale(Duration)),
                Age = as.vector(scale(Age)))


## Mixed-Model

Prepare data

Find frequent words (N > 5) that occur in both ENS and CHN data (WARNING: based on words determined to be shared in the ENS train, ENS test and CHN data for the MuPDARF)


In [ ]:
nswords <- c("bad", "be", "best", "big", "book", "books", "boss", "but", "did", "do", "due", "gas", "get", "gets", "good", "got", "guess", "had", "has", "he", "his", "keep", "kids", "other", "past", "pub", "put", "said", "see", "she", "sit", "step", "stop", "stud", "teeth", "that", "this", "too", "took", "tough", "two", "was", "we", "what", "who", "wish", "with")



process data and make it ready for GLMM



In [ ]:
mdat <- bdat %>%
  dplyr::filter(Vowel == "i"|Vowel == "I"|Vowel == "u"|Vowel == "U"|Vowel == "E"|Vowel == "{") %>%
  dplyr::select(-Vowel, -TargetVariety, -edist, -barkF1, -barkF2, -lobF1, 
                -lobF2, -normF1, -normF2, -cF1, -cF2, -ED, -WordType, -freq, -file, -id, -fspeaker) %>%
  dplyr::rename(Vowel = vowel) %>%
  dplyr::mutate(Word = ifelse(Word %in% nswords, Word, "other")) %>%
  dplyr::mutate_if(is.character, factor)
  # inspect
str(mdat)


Baseline model



In [ ]:
# set options
options(contrasts  =c("contr.treatment", "contr.poly"))
mdat.dist <- datadist(mdat)
options(datadist = "mdat.dist")
# generate initial minimal regression model 
# baseline model glm
ma = glmer(Duration ~ (1 | Word) + (1|Speaker), family = gaussian, data = mdat) 
# inspect results
summary(ma)
# inspect 
sjPlot::tab_model(ma)


Model fitting



In [ ]:
# wrapper function for linear mixed-models
glmer.glmulti <- function(formula, data, random="",...){
  lmer(paste(deparse(formula),random),  data=data, ...)
}
# define formular
form_glmulti = as.formula(paste("Duration ~  Vowel + type +  Gender + WordClass"))


Extract best 5 models.



In [ ]:
library(glmulti)
# multi selection for glmer
mfit <- glmulti(form_glmulti, random="+(1|Speaker)+(1|Word)", 
                data = mdat, method = "h", fitfunc = glmer.glmulti,  includeobjects = T,
                crit = "aic", intercept = TRUE, marginality = FALSE, level = 2)


After 50 models:
Best model: Duration~1+Vowel+type+Gender+type:Vowel+Gender:Vowel
Best model: Duration~1+Vowel+type+Gender+type:Vowel+Gender:Vowel+Gender:type
Crit= 15875.9439868615
Mean crit= 16152.5991758241

After 100 models:
Best model: Duration~1+Vowel+type+Gender+WordClass+Gender:Vowel
Best model: Duration~1+Vowel+type+Gender+WordClass+Gender:Vowel+WordClass:Vowel
Best model: Duration~1+Vowel+type+Gender+WordClass+Gender:Vowel+WordClass:type
Best model: Duration~1+Vowel+type+Gender+WordClass+Gender:Vowel+WordClass:Vowel+WordClass:type
Best model: Duration~1+Vowel+type+Gender+WordClass+Gender:Vowel+WordClass:Gender
Best model: Duration~1+Vowel+type+Gender+WordClass+Gender:Vowel+WordClass:Vowel+WordClass:Gender
Best model: Duration~1+Vowel+type+Gender+WordClass+Gender:Vowel+WordClass:type+WordClass:Gender
Best model: Duration~1+Vowel+type+Gender+WordClass+Gender:Vowel+WordClass:Vowel+WordClass:type+WordClass:Gender
Crit= 15869.1516948654
Mean crit= 16099.4464548991
Completed.


In [ ]:
mb <- lmer(Duration ~ (1 | Word) + (1|Speaker) +
             type + Vowel + Gender + WordClass + Gender:Vowel,
           data = mdat)
# inspect 
sjPlot::tab_model(ma, mb)


Visualize effects
 


In [ ]:
sjPlot::plot_model(mb, type = "pred", terms = c("Vowel", "type")) +
  scale_color_manual(values = c("lightgray", "orange")) +
  theme_bw() +
  labs(title = "", y = "Predicted duration", x = "Speaker type")
ggsave2(here::here("images", "lmer_type.png"), width = 4, height = 3)


In [ ]:
sjPlot::plot_model(mb, type = "pred", terms = c("Vowel", "Gender")) +
  theme_bw() +
  labs(title = "", y = "Predicted duration")
ggsave2(here::here("images", "lmer_vowelf2.png"), width = 4, height = 3)


## Overlap

Check density


In [ ]:
wordplot3 <- function(fdat, vwl1, vwl2){
  plt <- fdat %>% 
    dplyr::rename(label = Vowel)  %>%
    dplyr::filter(label == vwl1 | label == vwl2) %>%
    dplyr::group_by(Word, label) %>%
    dplyr::mutate(meanF2 = mean(lobF2),
                  meanF1 = mean(lobF1)) %>%
    dplyr::ungroup() %>%
    dplyr::group_by(label) %>%
    dplyr::mutate(cF2 = mean(lobF2),
                  cF1 = mean(lobF1)) %>%
    # plot
    ggplot(aes(x = lobF2, y = lobF1)) +
  stat_density_2d(geom = "polygon",
                  aes(alpha = ..level.., fill = label), bins = 8)  +
    facet_grid( ~ type) +
    scale_x_reverse(position = "top") + 
    scale_y_reverse(position = "right") + 
    #scale_fill_distiller(palette = "Blues", direction = 1) +
    geom_text(aes(x = meanF2, y = meanF1, 
                  label = Word, color = label), size = 4) +
    geom_text(aes(x = cF2, y = cF1, 
                  label = vowel), size = 6, color = "gray20") +
    theme_minimal() +
    theme(panel.grid.major = element_blank(), 
                  panel.grid.minor = element_blank(),
                  legend.position = "none") +
  scale_color_manual(values = c("orange3", "gray40")) +
  scale_fill_manual(values = c("orange", "gray")) +
    labs(x = "Formant 2 (Lobanov normalized)", y = "Formant 1 (Lobanov normalized)")
  return(plt)
  }


In [ ]:
pIi <- wordplot3(fdat = bdat, vwl1 = "I", vwl2 = "i")
ggsave(here::here("images", "pIi.png"), height = 3,  width = 5, dpi = 320)
pIi


In [ ]:
wordplot3 <- function(fdat, vwl1, vwl2){
  plt <- fdat %>% 
    dplyr::rename(label = Vowel)  %>%
    dplyr::filter(label == vwl1 | label == vwl2) %>%
    dplyr::group_by(Word, label) %>%
    dplyr::mutate(meanF2 = mean(lobF2),
                  meanF1 = mean(lobF1)) %>%
    dplyr::ungroup() %>%
    dplyr::group_by(label) %>%
    dplyr::mutate(cF2 = mean(lobF2),
                  cF1 = mean(lobF1)) %>%
    # plot
    ggplot(aes(x = lobF2, y = lobF1)) +
  stat_density_2d(geom = "polygon",
                  aes(alpha = ..level.., fill = label), bins = 8)  +
    facet_grid( ~ type) +
    scale_x_reverse(position = "top") + 
    scale_y_reverse(position = "right") + 
    #scale_fill_distiller(palette = "Blues", direction = 1) +
    geom_text(aes(x = meanF2, y = meanF1, 
                  label = Word, color = label), size = 4) +
    geom_text(aes(x = cF2, y = cF1, 
                  label = vowel), size = 6, color = "gray20") +
    theme_minimal() +
    theme(panel.grid.major = element_blank(), 
                  panel.grid.minor = element_blank(),
                  legend.position = "none") +
  scale_color_manual(values = c("red", "darkblue")) +
  scale_fill_manual(values = c("salmon", "lightblue")) +
    labs(x = "Formant 2 (Lobanov normalized)", y = "Formant 1 (Lobanov normalized)")
  return(plt)
  }


In [ ]:
pUu <- wordplot3(fdat = bdat, vwl1 = "U", vwl2 = "u")
ggsave(here::here("images", "pUu.png"), height = 3,  width = 5, dpi = 320)
pUu


In [ ]:
wordplot3 <- function(fdat, vwl1, vwl2){
  plt <- fdat %>% 
    dplyr::rename(label = Vowel)  %>%
    dplyr::filter(label == vwl1 | label == vwl2) %>%
    dplyr::group_by(Word, label) %>%
    dplyr::mutate(meanF2 = mean(lobF2),
                  meanF1 = mean(lobF1)) %>%
    dplyr::ungroup() %>%
    dplyr::group_by(label) %>%
    dplyr::mutate(cF2 = mean(lobF2),
                  cF1 = mean(lobF1)) %>%
    # plot
    ggplot(aes(x = lobF2, y = lobF1)) +
  stat_density_2d(geom = "polygon",
                  aes(alpha = ..level.., fill = label), bins = 8)  +
    facet_grid( ~ type) +
    scale_x_reverse(position = "top") + 
    scale_y_reverse(position = "right") + 
    #scale_fill_distiller(palette = "Blues", direction = 1) +
    geom_text(aes(x = meanF2, y = meanF1, 
                  label = Word, color = label), size = 4) +
    geom_text(aes(x = cF2, y = cF1, 
                  label = vowel), size = 6, color = "gray20") +
    theme_minimal() +
    theme(panel.grid.major = element_blank(), 
                  panel.grid.minor = element_blank(),
                  legend.position = "none") +
  scale_color_manual(values = c("green", "darkorchid4")) +
  scale_fill_manual(values = c("lightgreen", "darkorchid1")) +
    labs(x = "Formant 2 (Lobanov normalized)", y = "Formant 1 (Lobanov normalized)")
  return(plt)
  }


In [ ]:
pEe <- wordplot3(fdat = bdat, vwl1 = "{", vwl2 = "E")
ggsave(here::here("images", "pEe.png"), height = 3,  width = 5, dpi = 320)
pEe


## Bhattacharyya's affinity 

function for extracting Bhattacharyya's affinity by type and target variety


In [ ]:
exba <- function(data, section, target, vwl1, vwl2){
  ba <- data %>%
  ungroup() %>%
  dplyr::filter(type == section,
                TargetVariety == target,
                Vowel == vwl1 | Vowel == vwl2)

  ba_formants <- ba %>%  dplyr::select(lobF1, lobF2)
  # extract vowels
  ba_vowel <- ba %>%  dplyr::select(vowel)
  # spatial data frame
  ba_spdf <- SpatialPointsDataFrame(ba_formants, ba_vowel)
  # calculate Bhattacharyya's affinity
  ba_ba <- kerneloverlap(ba_spdf, method = "BA")
  # result
  return(ba_ba[1,2])
}


### I vs i: 



In [ ]:
# CHN
exba(data = bdat, section = "CHN", target = "AmE", vwl1 = "I", vwl2 = "i")
# ENS
exba(data = bdat, section = "ENS", target = "AmE", vwl1 = "I", vwl2 = "i")


### U vs u:

High-back 


In [ ]:
# CHN
exba(data = bdat, section = "CHN", target = "AmE", vwl1 = "U", vwl2 = "u")
# ENS
exba(data = bdat, section = "ENS", target = "AmE", vwl1 = "U", vwl2 = "u")


### E vs {

High-back 


In [ ]:
# CHN
exba(data = bdat, section = "CHN", target = "AmE", vwl1 = "{", vwl2 = "E")
# ENS
exba(data = bdat, section = "ENS", target = "AmE", vwl1 = "{", vwl2 = "E")


## Tabulation  of the data



In [ ]:
tb1 <- bdat %>%
  dplyr::filter(Vowel == "{"|Vowel == "E"|Vowel == "i"|Vowel == "I"|Vowel == "u"|Vowel == "U")  %>%
  dplyr::mutate(Vowel = dplyr::case_when(Vowel == "{" ~ "\u00E6",
                                         Vowel == "6" ~ "\u0250",
                                         Vowel == "e" ~ "\u0065",
                                         Vowel == "E" ~ "\u025B",
                                         Vowel == "i" ~ "\u0069",
                                         Vowel == "I" ~ "\u026A",
                                         Vowel == "Q" ~ "\u0252",
                                         Vowel == "u" ~ "\u0075",
                                         Vowel == "U" ~ "\u028A",
                                         Vowel == "V" ~ "\u028C",
                                         TRUE ~ Vowel))  %>%
  dplyr::ungroup() %>%
  dplyr::group_by(type) %>%
  dplyr::mutate(speakers = length(table(Speaker))) %>%
  dplyr::ungroup() %>%
  dplyr::group_by(type, Vowel) %>%
  dplyr::summarise(speakers = speakers,
                   obs = n()) %>%
  unique() %>%
  tidyr::spread(Vowel, obs) %>%
  dplyr::ungroup()  %>%
  adorn_totals("row")%>%
  adorn_totals("col") %>%
  dplyr::mutate(Total = Total-speakers)
# save
write.table(tb1, here::here("tables", "tb1_icame.txt"), sep = "\t")
# inspect
tb1


tabulate proficiency



In [ ]:
tb3 <- bdat %>%
  dplyr::ungroup() %>%
  dplyr::filter(type == "CHN")%>%
  dplyr::group_by(Proficiency, Gender) %>%
  dplyr::summarise(speakers = length(table(Speaker))) %>%
  tidyr::spread(Proficiency, speakers) %>%
  dplyr::ungroup()  %>%
  adorn_totals("row")%>%
  adorn_totals("col")
# save
write.table(tb3, here::here("tables", "tb3_icame.txt"), sep = "\t")
# inspect
tb3


In [ ]:
summary(bdat)



In [ ]:
summary(mdat)



## Outro



In [ ]:
# save tables
base::saveRDS(mdat, file = here::here("tables", "mdat.rda"))
base::saveRDS(bdat, file = here::here("tables", "bdat.rda"))


## Citation & Session Info

Schweinberger, Martin and Ruihua Yin. 2023. A Corpus-Based Acoustic Analysis of Monophthongal Vowels among Chinese Learners and Native Speakers of English. Brisbane: The University of Queensland, School of Languages and Cultures. 


In [ ]:
sessionInfo()

